In [ ]:
import pandas as pd
import numpy as np
from m16_mlutils.pipeline import CategoryEncoder, DataFrameSelector
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from m16_mlutils.datatools.evaluation import eval_summary

from numpy.random import seed

seed(42)

import os

HOME = os.getenv('HOME')

from pipelines import IsPunctuation, RelativeLocations, Reshaper, SentenceChunker

In [ ]:
chunker = SentenceChunker(HOME + '/stanford_nlp/stanford-postagger-full-2018-02-27/models/spanish-distsim.tagger',
                HOME + '/stanford_nlp/stanford-postagger-full-2018-02-27/stanford-postagger.jar')

df = chunker.transform(['¡CDMX a Bahamas $8,722! (Agrega 7 noches de hotel por $7,203)'])

In [ ]:
def get_pipeline():
    pipeline = joblib.load('data/i__estimate_pipeline.joblib') 
    return pipeline

classifier = get_pipeline()

In [ ]:
labels = classifier.predict(df)
df['labels'] = labels

In [ ]:
def label_data(sentence):
    df = chunker.transform([sentence])
    labels = classifier.predict(df)
    df['labels'] = labels
    destination_tokens = []
    origin_tokens = []
    price = -1
    for token, label in zip(df['token'].values, labels):
        if label == 'o':
            origin_tokens.append(token)
        elif label == 'd':
            destination_tokens.append(token)
        elif label == 'p':
            price = float(token.replace(',',''))
    return {
        'origin': ' '.join(origin_tokens),
        'destination': ' '.join(destination_tokens),
        'price': price
    }

label_data('¡CDMX, MTY y GDL a Liberia, Costa Rica $2,458! (Por $2,683 agrega 7 noches de hotel)')